In [12]:
import nltk
from nltk import word_tokenize, sent_tokenize
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
import jieba
%matplotlib inline

In [6]:
training_path = 'training.csv'
testing_path = 'testing.csv'

df_train = pd.read_csv(training_path, names=['class', 'descriptions'])

In [7]:
df_train.head()

,class,descriptions
0,2,合晟资产是一家专注于股票、债券等二级市场投资，为合格投资者提供专业资产管理服务的企业。公司业...
1,2,公司的主营业务为向中小微企业、个体工商户、农户等客户提供贷款服务，自设立以来主营业务未发生过变化。
2,1,公司立足于商业地产服务，致力于为商业地产开发、销售、运营全产业链提供一整套增值服务，业务覆盖...
3,2,公司经工商管理部门核准的经营范围为“投资咨询、经济信息咨询，企业管理咨询，品牌推广策划，公共...
4,2,该公司的主营业务为在中国境内(港、澳、台除外)开展保险代理销售，依托于自身的产品研究能力和专...


In [9]:
desc_str = df_train['descriptions'][0]
desc_str

'合晟资产是一家专注于股票、债券等二级市场投资，为合格投资者提供专业资产管理服务的企业。公司业务范围包括资产管理、投资咨询和投资顾问服务。公司管理的私募基金产品主要包括股票型、债券型资产管理计划或证券投资基金，管理总资产规模80亿元左右。根据中国证券投资基金业协会数据，公司管理的私募证券投资基金（顾问管理）类规模较大，公司管理规模处于50亿元以上的第一梯队。'

In [17]:
seg_list_cut_all = jieba.lcut(desc_str, cut_all=True)
seg_list_cut_all

['合',
 '晟',
 '资产',
 '是',
 '一家',
 '专注',
 '于',
 '股票',
 '',
 '',
 '债券',
 '等',
 '二级',
 '市场',
 '投资',
 '',
 '',
 '为',
 '合格',
 '投资',
 '投资者',
 '提供',
 '专业',
 '资产',
 '管理',
 '服务',
 '的',
 '企业',
 '',
 '',
 '公司',
 '司业',
 '业务',
 '业务范围',
 '范围',
 '围包',
 '包括',
 '资产',
 '管理',
 '',
 '',
 '投资',
 '咨询',
 '和',
 '投资',
 '顾问',
 '服务',
 '',
 '',
 '公司',
 '管理',
 '的',
 '私募',
 '基金',
 '产品',
 '主要',
 '包括',
 '股票',
 '型',
 '',
 '',
 '债券',
 '型',
 '资产',
 '管理',
 '计划',
 '或',
 '证券',
 '投资',
 '基金',
 '',
 '',
 '管理',
 '总资产',
 '资产',
 '规模',
 '80',
 '亿元',
 '左右',
 '',
 '',
 '根据',
 '中国',
 '证券',
 '投资',
 '基金',
 '基金业',
 '金业',
 '业协会',
 '协会',
 '数据',
 '',
 '',
 '公司',
 '管理',
 '的',
 '私募',
 '证券',
 '投资',
 '基金',
 '',
 '',
 '顾问',
 '管理',
 '',
 '',
 '类',
 '规模',
 '规模较',
 '较大',
 '',
 '',
 '公司',
 '管理',
 '规模',
 '处于',
 '50',
 '亿元',
 '以上',
 '的',
 '第一',
 '一梯',
 '梯队',
 '',
 '']

In [18]:
seg_list_cut = jieba.lcut(desc_str, cut_all=False)
seg_list_cut

['合晟',
 '资产',
 '是',
 '一家',
 '专注',
 '于',
 '股票',
 '、',
 '债券',
 '等',
 '二级',
 '市场',
 '投资',
 '，',
 '为',
 '合格',
 '投资者',
 '提供',
 '专业',
 '资产',
 '管理',
 '服务',
 '的',
 '企业',
 '。',
 '公司',
 '业务范围',
 '包括',
 '资产',
 '管理',
 '、',
 '投资',
 '咨询',
 '和',
 '投资',
 '顾问',
 '服务',
 '。',
 '公司',
 '管理',
 '的',
 '私募',
 '基金',
 '产品',
 '主要',
 '包括',
 '股票',
 '型',
 '、',
 '债券',
 '型',
 '资产',
 '管理',
 '计划',
 '或',
 '证券',
 '投资',
 '基金',
 '，',
 '管理',
 '总资产',
 '规模',
 '80',
 '亿元',
 '左右',
 '。',
 '根据',
 '中国',
 '证券',
 '投资',
 '基金业',
 '协会',
 '数据',
 '，',
 '公司',
 '管理',
 '的',
 '私募',
 '证券',
 '投资',
 '基金',
 '（',
 '顾问',
 '管理',
 '）',
 '类',
 '规模',
 '较大',
 '，',
 '公司',
 '管理',
 '规模',
 '处于',
 '50',
 '亿元',
 '以上',
 '的',
 '第一',
 '梯队',
 '。']

In [19]:
seg_list_search = jieba.lcut_for_search(desc_str)
seg_list_search

['合晟',
 '资产',
 '是',
 '一家',
 '专注',
 '于',
 '股票',
 '、',
 '债券',
 '等',
 '二级',
 '市场',
 '投资',
 '，',
 '为',
 '合格',
 '投资',
 '投资者',
 '提供',
 '专业',
 '资产',
 '管理',
 '服务',
 '的',
 '企业',
 '。',
 '公司',
 '业务',
 '范围',
 '业务范围',
 '包括',
 '资产',
 '管理',
 '、',
 '投资',
 '咨询',
 '和',
 '投资',
 '顾问',
 '服务',
 '。',
 '公司',
 '管理',
 '的',
 '私募',
 '基金',
 '产品',
 '主要',
 '包括',
 '股票',
 '型',
 '、',
 '债券',
 '型',
 '资产',
 '管理',
 '计划',
 '或',
 '证券',
 '投资',
 '基金',
 '，',
 '管理',
 '资产',
 '总资产',
 '规模',
 '80',
 '亿元',
 '左右',
 '。',
 '根据',
 '中国',
 '证券',
 '投资',
 '基金',
 '金业',
 '基金业',
 '协会',
 '数据',
 '，',
 '公司',
 '管理',
 '的',
 '私募',
 '证券',
 '投资',
 '基金',
 '（',
 '顾问',
 '管理',
 '）',
 '类',
 '规模',
 '较大',
 '，',
 '公司',
 '管理',
 '规模',
 '处于',
 '50',
 '亿元',
 '以上',
 '的',
 '第一',
 '梯队',
 '。']